In [191]:
# %pip install python-dotenv
# %pip install pandas
# %pip install requests
# %pip install pyshacl
# %pip install brickschema

In [97]:
import requests
import pandas as pd
from rdflib import Graph
import os
import pyshacl
import json
from pandas import DataFrame


In [98]:
prefixes = {
    'brick:': 'https://brickschema.org/schema/Brick#',
    'ref:': 'https://brickschema.org/schema/Brick/ref#',
    'ref-ext:': 'https://w3id.org/b4b/schemes/ext/ref-ext#',
    'sba:': 'https://w3id.org/b4b/schemes/algorithms/sba#', 
    'mpc:': 'https://w3id.org/b4b/schemes/algorithms/mpc#', 
    'forc:': 'https://w3id.org/b4b/schemes/algorithms/forc#', 
    'brick-ev:': '<https://w3id.org/b4b/schemes/ext/brick-ev#>',
    'brick-ext:': '<https://w3id.org/b4b/schemes/ext/brick-ext#>',
}

In [99]:
def do_prefixes(data):
    if type(data) is list:
        for i, item in enumerate(data):
            for prefix, uri in prefixes.items():
                if item.startswith(uri):
                    data[i] = item.replace(uri, prefix)
                    break

    elif type(data) is DataFrame:
        for prefix, uri in prefixes.items():
            data.columns = [col.replace(uri, prefix) for col in data.columns]
    return data

In [100]:
def semantic_query(endpoint, query, headers=None, auth=None, infer=False):   
    """
    Executes a SPARQL query against the specified endpoint.
    
    :param endpoint: The SPARQL endpoint URL.
    :param query: The SPARQL query string.
    :param headers: Optional headers for the request.
    :param auth: Optional authentication tuple (username, password).
    :return: The response from the SPARQL endpoint.
    """
    headers = {
    "Content-Type": "application/sparql-query",
    "Accept": "application/json"}

    auth = ("lchamari", "lchamari")

    query_params = {
    'query': query,
    'infer': infer}

    response = requests.get(endpoint, params=query_params, headers=headers, auth=auth)
    if response.status_code != 200:
        raise Exception(f"SPARQL query failed with status code {response.status_code}: {response.text}")
    try:
        response_json = response.json()
    except json.JSONDecodeError:
        raise Exception("Failed to parse response from SPARQL endpoint. Response content: " + response.text)
    response_graph = Graph()
    response_graph.parse(data=response.text, format='json-ld')
    return response_json




# Querying Schemas

In [173]:
sparql_endpoint = 'https://graphdb.bimsim.app/repositories/Smart_charging_MPC_Schemas'

## Forecast Inputs

In [174]:
# sparql_endpoint = 'https://graphdb.bimsim.app/repositories/breda_plus_schemas'

query = """
PREFIX ref-ext: <https://w3id.org/b4b/schemes/ext/ref-ext#>
PREFIX ref: <https://brickschema.org/schema/Brick/ref#>
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX sba: <https://w3id.org/b4b/schemes/algorithms/sba#> 
PREFIX mpc: <https://w3id.org/b4b/schemes/algorithms/mpc#> 
PREFIX forc: <https://w3id.org/b4b/schemes/algorithms/forc#> 

select ?input ?tsref where {
    ?s ?p ?input .
    FILTER(?s=sba:LoadForecastAlgorithm)
    FILTER(?p=sba:hasInput)
    ?input ref:hasExternalReference ?arr .
    ?arr a ?tsref .
}
    """
result = semantic_query(sparql_endpoint, query, infer=True)
forc_inputs = [binding['input']['value'] for binding in result['results']['bindings']]
# forc_inputs=do_prefixes(inputs)
forc_inputs


['https://brickschema.org/schema/Brick#Building_Electrical_Meter',
 'https://brickschema.org/schema/Brick#Building_Electrical_Meter',
 'https://brickschema.org/schema/Brick#Building_Electrical_Meter',
 'https://brickschema.org/schema/Brick#Building_Electrical_Meter',
 'https://brickschema.org/schema/Brick#Building_Electrical_Meter',
 'https://brickschema.org/schema/Brick#Building_Electrical_Meter',
 'https://brickschema.org/schema/Brick#Outside_Air_Temperature_Sensor',
 'https://brickschema.org/schema/Brick#Outside_Air_Temperature_Sensor',
 'https://brickschema.org/schema/Brick#Outside_Air_Temperature_Sensor',
 'https://brickschema.org/schema/Brick#Outside_Air_Temperature_Sensor',
 'https://brickschema.org/schema/Brick#Outside_Air_Temperature_Sensor',
 'https://brickschema.org/schema/Brick#Outside_Air_Temperature_Sensor',
 'https://brickschema.org/schema/Brick#Wind_Speed_Sensor',
 'https://brickschema.org/schema/Brick#Wind_Speed_Sensor',
 'https://brickschema.org/schema/Brick#Wind_Spee

## MPC constraints

In [175]:
q2 = """
PREFIX ref-ext: <https://w3id.org/b4b/schemes/ext/ref-ext#>
PREFIX brick-ev: <https://w3id.org/b4b/schemes/ext/brick-ev#>
PREFIX brick-ext: <https://w3id.org/b4b/schemes/ext/brick-ext#>
PREFIX ref: <https://brickschema.org/schema/Brick/ref#>
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX sba: <https://w3id.org/b4b/schemes/algorithms/sba#> 
PREFIX mpc: <https://w3id.org/b4b/schemes/algorithms/mpc#> 
PREFIX forc: <https://w3id.org/b4b/schemes/algorithms/forc#> 

select ?input ?tsref where {
    ?s ?p ?input .
    FILTER(?s=mpc:Smart_Charging_MPC_Constraints_Algorithm)
    FILTER(?p=sba:hasInput)
}
    """
result = semantic_query(sparql_endpoint, q2, infer=True)
constraints_inputs = [binding['input']['value'] for binding in result['results']['bindings']]
# constraints_inputs=do_prefixes(inputs)
constraints_inputs

['https://w3id.org/b4b/schemes/ext/brick-ev#Minimum_Charging_Current_Setpoint',
 'https://w3id.org/b4b/schemes/ext/brick-ev#Maximum_Charging_Current_Setpoint']

## Feedback Inputs

In [176]:
q3 = """
PREFIX ref-ext: <https://w3id.org/b4b/schemes/ext/ref-ext#>
PREFIX brick-ev: <https://w3id.org/b4b/schemes/ext/brick-ev#>
PREFIX brick-ext: <https://w3id.org/b4b/schemes/ext/brick-ext#>
PREFIX ref: <https://brickschema.org/schema/Brick/ref#>
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX sba: <https://w3id.org/b4b/schemes/algorithms/sba#> 
PREFIX mpc: <https://w3id.org/b4b/schemes/algorithms/mpc#> 
PREFIX forc: <https://w3id.org/b4b/schemes/algorithms/forc#> 

select DISTINCT ?input ?tsref where {
    ?s ?p ?input .
    FILTER(?s=mpc:EVRealtimeFeedbackAlgorithm)
    FILTER(?p=sba:hasInputData)
    ?input ref:hasExternalReference ?arr .
    ?arr a ?tsref .
}
    """
result = semantic_query(sparql_endpoint, q3, infer=True)
feedback_inputs = [binding['input']['value'] for binding in result['results']['bindings']]
# feedback_inputs=do_prefixes(inputs)
feedback_inputs

['https://w3id.org/b4b/schemes/ext/brick-ev#EV_Charging_Station_ID',
 'https://w3id.org/b4b/schemes/ext/brick-ev#EV_Charging_Station_ID',
 'https://w3id.org/b4b/schemes/ext/brick-ev#EV_Arrival_Time',
 'https://w3id.org/b4b/schemes/ext/brick-ev#EV_Arrival_Time',
 'https://w3id.org/b4b/schemes/ext/brick-ev#EV_Departure_Time',
 'https://w3id.org/b4b/schemes/ext/brick-ev#EV_Departure_Time',
 'https://w3id.org/b4b/schemes/ext/brick-ev#EV_Arrival_SOC',
 'https://w3id.org/b4b/schemes/ext/brick-ev#EV_Arrival_SOC',
 'https://w3id.org/b4b/schemes/ext/brick-ev#EV_Battery_Size',
 'https://w3id.org/b4b/schemes/ext/brick-ev#EV_Battery_Size',
 'https://w3id.org/b4b/schemes/ext/brick-ev#Vehicle_Full_Status',
 'https://w3id.org/b4b/schemes/ext/brick-ev#Vehicle_Full_Status',
 'https://w3id.org/b4b/schemes/ext/brick-ev#Charging_Session_Status',
 'https://w3id.org/b4b/schemes/ext/brick-ev#Charging_Session_Status',
 'https://w3id.org/b4b/schemes/ext/brick-ev#EV_Electric_Energy_Meter',
 'https://w3id.org/b4

## Static Parameter estimation

In [177]:
q4 = """
PREFIX ref-ext: <https://w3id.org/b4b/schemes/ext/ref-ext#>
PREFIX brick-ev: <https://w3id.org/b4b/schemes/ext/brick-ev#>
PREFIX brick-ext: <https://w3id.org/b4b/schemes/ext/brick-ext#>
PREFIX ref: <https://brickschema.org/schema/Brick/ref#>
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX sba: <https://w3id.org/b4b/schemes/algorithms/sba#> 
PREFIX mpc: <https://w3id.org/b4b/schemes/algorithms/mpc#> 
PREFIX forc: <https://w3id.org/b4b/schemes/algorithms/forc#> 

select ?input ?tsref where {
    ?s ?p ?input .
    FILTER(?s=mpc:StaticParameterEstimationAlgorithm)
    FILTER(?p=sba:hasInput)
}
    """
result = semantic_query(sparql_endpoint, q4, infer=True)
static_param_inputs = [binding['input']['value'] for binding in result['results']['bindings']]
# static_param_inputs=do_prefixes(inputs)
static_param_inputs

['https://w3id.org/b4b/schemes/ext/brick-ev#Charger_Voltage_Rating',
 'https://w3id.org/b4b/schemes/ext/brick-ev#Charger_Power_Rating',
 'https://w3id.org/b4b/schemes/ext/brick-ext#Power_Factor',
 'https://w3id.org/b4b/schemes/algorithms/sba#BuildingPowerContract']

## Dynamic Parameter estimation

In [178]:
q5 = """
PREFIX ref-ext: <https://w3id.org/b4b/schemes/ext/ref-ext#>
PREFIX brick-ev: <https://w3id.org/b4b/schemes/ext/brick-ev#>
PREFIX brick-ext: <https://w3id.org/b4b/schemes/ext/brick-ext#>
PREFIX ref: <https://brickschema.org/schema/Brick/ref#>
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX sba: <https://w3id.org/b4b/schemes/algorithms/sba#> 
PREFIX mpc: <https://w3id.org/b4b/schemes/algorithms/mpc#> 
PREFIX forc: <https://w3id.org/b4b/schemes/algorithms/forc#> 

select ?input ?tsref where {
    ?s ?p ?input .
    FILTER(?s=mpc:DynamicParameterEstimationAlgorithm)
    FILTER(?p=sba:hasInput)
}
    """
result = semantic_query(sparql_endpoint, q5, infer=True)
dynamic_param_inputs = [binding['input']['value'] for binding in result['results']['bindings']]
# dynamic_param_inputs=do_prefixes(inputs)
dynamic_param_inputs

['https://w3id.org/b4b/schemes/ext/brick-ev#EV_Electric_Energy_Meter',
 'https://w3id.org/b4b/schemes/ext/brick-ev#EV_Electric_Power_Sensor']

## Estate estimation

In [179]:
q6 = """
PREFIX ref-ext: <https://w3id.org/b4b/schemes/ext/ref-ext#>
PREFIX brick-ev: <https://w3id.org/b4b/schemes/ext/brick-ev#>
PREFIX brick-ext: <https://w3id.org/b4b/schemes/ext/brick-ext#>
PREFIX ref: <https://brickschema.org/schema/Brick/ref#>
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX sba: <https://w3id.org/b4b/schemes/algorithms/sba#> 
PREFIX mpc: <https://w3id.org/b4b/schemes/algorithms/mpc#> 
PREFIX forc: <https://w3id.org/b4b/schemes/algorithms/forc#> 

select ?input ?tsref where {
    ?s ?p ?input .
    FILTER(?s=mpc:EVStateEstimationAlgorithm)
    FILTER(?p=sba:hasInput)
}
    """
result = semantic_query(sparql_endpoint, q6, infer=True)
state_inputs = [binding['input']['value'] for binding in result['results']['bindings']]
# state_inputs=do_prefixes(inputs)
state_inputs

['https://w3id.org/b4b/schemes/ext/brick-ev#EV_Charging_Station_ID',
 'https://w3id.org/b4b/schemes/ext/brick-ev#EV_Arrival_Time',
 'https://w3id.org/b4b/schemes/ext/brick-ev#EV_Departure_Time',
 'https://w3id.org/b4b/schemes/ext/brick-ev#EV_Arrival_SOC',
 'https://w3id.org/b4b/schemes/ext/brick-ev#EV_Battery_Size',
 'https://w3id.org/b4b/schemes/ext/brick-ev#Vehicle_Full_Status',
 'https://w3id.org/b4b/schemes/ext/brick-ev#Charging_Session_Status',
 'https://w3id.org/b4b/schemes/ext/brick-ev#EV_Electric_Energy_Meter',
 'https://w3id.org/b4b/schemes/ext/brick-ev#EV_Charging_Session_Visual_Number',
 'https://w3id.org/b4b/schemes/ext/brick-ev#EV_Load_Current_Sensor']

## Send setpoints

In [180]:
q7 = """
PREFIX ref-ext: <https://w3id.org/b4b/schemes/ext/ref-ext#>
PREFIX brick-ev: <https://w3id.org/b4b/schemes/ext/brick-ev#>
PREFIX brick-ext: <https://w3id.org/b4b/schemes/ext/brick-ext#>
PREFIX ref: <https://brickschema.org/schema/Brick/ref#>
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX sba: <https://w3id.org/b4b/schemes/algorithms/sba#> 
PREFIX mpc: <https://w3id.org/b4b/schemes/algorithms/mpc#> 
PREFIX forc: <https://w3id.org/b4b/schemes/algorithms/forc#> 

select ?input ?tsref where {
    ?s ?p ?input .
    FILTER(?s=mpc:EVChargingCurrentSetpointCommunication)
    FILTER(?p=sba:hasInput)
}
    """
result = semantic_query(sparql_endpoint, q7, infer=True)
setpoint_inputs = [binding['input']['value'] for binding in result['results']['bindings']]
# setpoint_inputs=do_prefixes(inputs)
setpoint_inputs

['https://w3id.org/b4b/schemes/ext/brick-ev#MinimumEVChargingCurrentSetpoint',
 'https://w3id.org/b4b/schemes/ext/brick-ev#MaximumEVChargingCurrentSetpoint',
 'https://w3id.org/b4b/schemes/ext/brick-ev#EVChargingCurrentSetpoint']

# Check Availability

In [181]:
def ask_query(building_sparql_endpoint, inputs):

    construct_query = """
    CONSTRUCT { ?s ?p ?o }
    WHERE { ?s ?p ?o }
    """
    query_params = {
        'query': construct_query,
        'infer': 'true' 
    }
    headers = {
    "Content-Type": "application/sparql-query",
    "Accept": "application/json"
    }
    auth = ("lchamari", "lchamari")

    response = requests.get(building_sparql_endpoint, params=query_params,  auth=auth)

    if response.status_code == 200:
        response_text = response.text
        building2_data_graph= Graph()
        building2_data_graph.parse(data=response_text, format="turtle")
        print(f"Loaded {len(building2_data_graph)} tripples")

    else:
        print(f"Error: {response.status_code} - {response.reason}")


    status_dict = {}
    for point in inputs:
            if point.startswith("http"):
                class_expr = f"<{point}>"
            else:
                class_expr = point
            ask_query = f"""
            PREFIX brick-ext: <https://w3id.org/b4b/schemes/ext/brick-ext#>
            PREFIX ref-ext: <https://w3id.org/b4b/schemes/ext/ref-ext#>
            PREFIX ref: <https://brickschema.org/schema/Brick/ref#>
            PREFIX brick: <https://brickschema.org/schema/Brick#>
            PREFIX sba: <https://w3id.org/b4b/schemes/algorithms/sba#> 
            PREFIX mpc: <https://w3id.org/b4b/schemes/algorithms/mpc#> 
            PREFIX forc: <https://w3id.org/b4b/schemes/algorithms/forc#> 
            PREFIX brick-ev: <https://w3id.org/b4b/schemes/ext/brick-ev#>
            ASK WHERE {{
                ?s a {class_expr}
            }}
            """
            res = building2_data_graph.query(ask_query)
            staus=res.askAnswer
            status_dict[point] = staus
    df=pd.DataFrame(status_dict.items(), columns=['Data Point', 'Status'])
    df.sort_values(by=['Status'], ascending=False)
    return df

### Breda

In [182]:
# breda_sparql_endpoint='https://graphdb.bimsim.app/repositories/breda'
breda_sparql_endpoint='https://graphdb.bimsim.app/repositories/building_v1'

In [183]:
df=ask_query(breda_sparql_endpoint, inputs=forc_inputs)
df

Loaded 719 tripples


,Data Point,Status
0,https://brickschema.org/schema/Brick#Building_...,True
1,https://brickschema.org/schema/Brick#Outside_A...,False
2,https://brickschema.org/schema/Brick#Wind_Spee...,True
3,https://brickschema.org/schema/Brick#Solar_Rad...,True
4,https://w3id.org/b4b/schemes/algorithms/forc#O...,True
5,https://w3id.org/b4b/schemes/algorithms/forc#W...,True
6,https://w3id.org/b4b/schemes/algorithms/forc#S...,True


In [184]:
df=ask_query(breda_sparql_endpoint, inputs=constraints_inputs)
df

Loaded 719 tripples


,Data Point,Status
0,https://w3id.org/b4b/schemes/ext/brick-ev#Mini...,True
1,https://w3id.org/b4b/schemes/ext/brick-ev#Maxi...,True


In [185]:
df=ask_query(breda_sparql_endpoint, inputs=feedback_inputs)
df

Loaded 719 tripples


,Data Point,Status
0,https://w3id.org/b4b/schemes/ext/brick-ev#EV_C...,True
1,https://w3id.org/b4b/schemes/ext/brick-ev#EV_A...,True
2,https://w3id.org/b4b/schemes/ext/brick-ev#EV_D...,False
3,https://w3id.org/b4b/schemes/ext/brick-ev#EV_A...,True
4,https://w3id.org/b4b/schemes/ext/brick-ev#EV_B...,True
5,https://w3id.org/b4b/schemes/ext/brick-ev#Vehi...,True
6,https://w3id.org/b4b/schemes/ext/brick-ev#Char...,True
7,https://w3id.org/b4b/schemes/ext/brick-ev#EV_E...,True
8,https://w3id.org/b4b/schemes/ext/brick-ev#EV_C...,True


In [186]:
df=ask_query(breda_sparql_endpoint, inputs=static_param_inputs)
df

Loaded 719 tripples


,Data Point,Status
0,https://w3id.org/b4b/schemes/ext/brick-ev#Char...,True
1,https://w3id.org/b4b/schemes/ext/brick-ev#Char...,True
2,https://w3id.org/b4b/schemes/ext/brick-ext#Pow...,True
3,https://w3id.org/b4b/schemes/algorithms/sba#Bu...,False


In [187]:
df=ask_query(breda_sparql_endpoint, inputs=dynamic_param_inputs)
df

Loaded 719 tripples


,Data Point,Status
0,https://w3id.org/b4b/schemes/ext/brick-ev#EV_E...,True
1,https://w3id.org/b4b/schemes/ext/brick-ev#EV_E...,True


In [188]:
df=ask_query(breda_sparql_endpoint, inputs=state_inputs)
df

Loaded 719 tripples


,Data Point,Status
0,https://w3id.org/b4b/schemes/ext/brick-ev#EV_C...,True
1,https://w3id.org/b4b/schemes/ext/brick-ev#EV_A...,True
2,https://w3id.org/b4b/schemes/ext/brick-ev#EV_D...,False
3,https://w3id.org/b4b/schemes/ext/brick-ev#EV_A...,True
4,https://w3id.org/b4b/schemes/ext/brick-ev#EV_B...,True
5,https://w3id.org/b4b/schemes/ext/brick-ev#Vehi...,True
6,https://w3id.org/b4b/schemes/ext/brick-ev#Char...,True
7,https://w3id.org/b4b/schemes/ext/brick-ev#EV_E...,True
8,https://w3id.org/b4b/schemes/ext/brick-ev#EV_C...,True
9,https://w3id.org/b4b/schemes/ext/brick-ev#EV_L...,True


In [189]:
df=ask_query(breda_sparql_endpoint, inputs=setpoint_inputs)
df

Loaded 719 tripples


,Data Point,Status
0,https://w3id.org/b4b/schemes/ext/brick-ev#Mini...,False
1,https://w3id.org/b4b/schemes/ext/brick-ev#Maxi...,False
2,https://w3id.org/b4b/schemes/ext/brick-ev#EVCh...,False


###  Configurations

In [190]:
sparql_endpoint = 'https://graphdb.bimsim.app/repositories/building_v1'

In [191]:
def do_config(query):
    result = semantic_query(sparql_endpoint, query, infer=True)
    vars = result['head']['vars']
    bindings = result['results']['bindings']
    data = []
    for binding in bindings:
        row = {var: binding[var]['value'] for var in vars}
        data.append(row)
    df = pd.DataFrame(data)
    return df

### Constraints

In [192]:
query = """
PREFIX ref-ext: <https://w3id.org/b4b/schemes/ext/ref-ext#>
PREFIX ref: <https://brickschema.org/schema/Brick/ref#>
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX brick-ev: <https://w3id.org/b4b/schemes/ext/brick-ev#>
PREFIX sba: <https://w3id.org/b4b/schemes/algorithms/sba#> 
PREFIX mpc: <https://w3id.org/b4b/schemes/algorithms/mpc#> 
PREFIX forc: <https://w3id.org/b4b/schemes/algorithms/forc#> 

SELECT DISTINCT ?Imin ?Imax
WHERE { 
    
?s a brick:Electric_Vehicle_Charging_Port .
?s  brick-ev:hasMinimumChargingCurrent ?Iminref.
?Iminref brick:value ?Imin .

?s a brick:Electric_Vehicle_Charging_Port .
?s  brick-ev:hasMaximumChargingCurrent ?Imaxref.
?Imaxref brick:value ?Imax .
}
    """
df = do_config(query)
df

,Imin,Imax
0,6,16


### Static params

In [193]:
query = """
PREFIX bldg1: <urn:building_kropman#>
PREFIX brick-ext: <https://w3id.org/b4b/schemes/ext/brick-ext#>
PREFIX bot: <https://w3id.org/bot#>
PREFIX ref-ext: <https://w3id.org/b4b/schemes/ext/ref-ext#>
PREFIX ref: <https://brickschema.org/schema/Brick/ref#>
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX brick-ev: <https://w3id.org/b4b/schemes/ext/brick-ev#>
PREFIX sba: <https://w3id.org/b4b/schemes/algorithms/sba#> 
PREFIX mpc: <https://w3id.org/b4b/schemes/algorithms/mpc#> 
PREFIX forc: <https://w3id.org/b4b/schemes/algorithms/forc#> 

SELECT DISTINCT ?chrgStId ?PowerRating ?Powerunits ?Vcharger ?Vunits ?BuildingContract ?BuildingContractUnits 
WHERE { 
   
?s a brick:Electric_Vehicle_Charging_Port .
?s brick:hasPoint ?chrgSt.
?s brick-ev:hasChargerPowerRating ?Prating .
?Prating a  brick-ev:Charger_Power_Rating .
?Prating brick:value ?PowerRating .
?Prating brick:hasUnit ?Powerunits .

?s a brick:Electric_Vehicle_Charging_Port .
?s brick-ev:hasChargerVoltageRating ?Vcharge.
?Vcharge a brick-ev:Charger_Voltage_Rating .
?Vcharge brick:value ?Vcharger .
?Vcharge brick:hasUnit ?Vunits .

?b a bot:Building .
?b brick-ext:hasBuildingPowerContract ?o .
?o a brick-ext:BuildingPowerContract .
?o brick:value ?BuildingContract .
?o brick:hasUnit ?BuildingContractUnits .

?chrgSt a brick-ev:EV_Charging_Station_ID .
?chrgSt  ref:hasExternalReference ?chrgStExt .
?chrgStExt ref-ext:hasRealtimeTimeseriesId ?chrgStId .

}
    """

df = do_config(query)
df

,chrgStId,PowerRating,Powerunits,Vcharger,Vunits,BuildingContract,BuildingContractUnits
0,1,11,http://qudt.org/vocab/unit/KiloW,230,http://qudt.org/vocab/unit/Volt,61,http://qudt.org/vocab/unit/KiloW
1,2,11,http://qudt.org/vocab/unit/KiloW,230,http://qudt.org/vocab/unit/Volt,61,http://qudt.org/vocab/unit/KiloW
2,3,11,http://qudt.org/vocab/unit/KiloW,230,http://qudt.org/vocab/unit/Volt,61,http://qudt.org/vocab/unit/KiloW
3,4,11,http://qudt.org/vocab/unit/KiloW,230,http://qudt.org/vocab/unit/Volt,61,http://qudt.org/vocab/unit/KiloW


### Dynamic params

In [194]:
query = """
PREFIX ref-ext: <https://w3id.org/b4b/schemes/ext/ref-ext#>
PREFIX ref: <https://brickschema.org/schema/Brick/ref#>
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX brick-ev: <https://w3id.org/b4b/schemes/ext/brick-ev#>
PREFIX sba: <https://w3id.org/b4b/schemes/algorithms/sba#> 
PREFIX mpc: <https://w3id.org/b4b/schemes/algorithms/mpc#> 
PREFIX forc: <https://w3id.org/b4b/schemes/algorithms/forc#> 

SELECT DISTINCT  ?chrgStId ?EVPower ?EVEnergy

WHERE {
  
?s a brick:Electric_Vehicle_Charging_Port .
?s brick:hasPoint ?chrgSt, ?evEnerMeter.
?evEnerMeter a  brick-ev:EV_Electric_Energy_Meter .
?evEnerMeter ref:hasExternalReference ?evEngExt .
?evEngExt ref-ext:hasHistoricalTimeseriesId ?EVEnergy .

?s a brick:Electric_Vehicle_Charging_Port .
?s brick:hasPoint ?chrgSt, ?evPowerMeter.
?evPowerMeter a brick-ev:EV_Electric_Power_Sensor .
?evPowerMeter ref:hasExternalReference ?evPExt .
?evPExt ref-ext:hasHistoricalTimeseriesId ?EVPower .

?chrgSt a brick-ev:EV_Charging_Station_ID .
?chrgSt  ref:hasExternalReference ?chrgStExt .
?chrgStExt ref-ext:hasRealtimeTimeseriesId ?chrgStId .

}
    """

df = do_config(query)
df

,chrgStId,EVPower,EVEnergy
0,1,56b56a995eae7ac5e331e115b3ad9a708.AVERAGE_POWE...,759ca9cc4f3bf8f552084d67f25d45899
1,2,cd8afb76be06cc43e288041260df100eb.AVERAGE_POWE...,1cb94c6d3ecd7c90b08317ad11121a31c
2,3,9fd044a2806b252458d24a054ecf0f145.AVERAGE_POWE...,f10dc57115f3546ff2ccb745362c73c96
3,4,737bf181efc276eec62f75bc34a171dd2.AVERAGE_POWE...,83a9cd04a4852e57bc3918762d1a049f3


### Realtime feedback

In [195]:
query = """
PREFIX ref-ext: <https://w3id.org/b4b/schemes/ext/ref-ext#>
PREFIX ref: <https://brickschema.org/schema/Brick/ref#>
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX brick-ev: <https://w3id.org/b4b/schemes/ext/brick-ev#>
PREFIX sba: <https://w3id.org/b4b/schemes/algorithms/sba#> 
PREFIX mpc: <https://w3id.org/b4b/schemes/algorithms/mpc#> 
PREFIX forc: <https://w3id.org/b4b/schemes/algorithms/forc#> 

SELECT DISTINCT ?chrgStId ?timeArrId ?timeDeptId ?arrivalSOCId ?evBatId ?vFullId ?chrgStatusId ?energyId ?visualNumId
where { 
    
?s a brick:Electric_Vehicle_Charging_Port .
#?s eva:physical_reference "2" .
?s brick:hasPoint ?chrgSt, ?timeArr, ?timeDept, ?arrivalSOC, ?evBat, ?vFull, ?chrgStatus, ?energy, ?visualNum .
    
?chrgSt a brick-ev:EV_Charging_Station_ID .
?chrgSt  ref:hasExternalReference ?chrgStExt .
?chrgStExt ref-ext:hasRealtimeTimeseriesId ?chrgStId .
        
?timeArr a brick-ev:EV_Arrival_Time .
?timeArr  ref:hasExternalReference ?timeArrExt .
?timeArrExt ref-ext:hasRealtimeTimeseriesId ?timeArrId .
    
?timeDept a brick-ev:EV_Departure_Time .
?timeDept  ref:hasExternalReference ?timeDeptExt .
?timeDeptExt ref-ext:hasRealtimeTimeseriesId ?timeDeptId .
    
?arrivalSOC a brick-ev:EV_Arrival_SOC .
?arrivalSOC  ref:hasExternalReference ?arrivalSOCExt .
?arrivalSOCExt ref-ext:hasRealtimeTimeseriesId ?arrivalSOCId .

?evBat a brick-ev:EV_Battery_Size .
?evBat  ref:hasExternalReference ?evBatExt .
?evBatExt ref-ext:hasRealtimeTimeseriesId ?evBatId .

?vFull a brick-ev:Vehicle_Full_Status .
?vFull  ref:hasExternalReference ?vFullExt .
?vFullExt ref-ext:hasRealtimeTimeseriesId ?vFullId .

?chrgStatus a brick-ev:Charging_Session_Status .
?chrgStatus  ref:hasExternalReference ?chrgStatusExt .
?chrgStatusExt ref-ext:hasRealtimeTimeseriesId ?chrgStatusId .
        
?energy a brick-ev:EV_Electric_Energy_Meter .
?energy  ref:hasExternalReference ?energyExt .
?energyExt ref-ext:hasHistoricalTimeseriesId ?energyId .

?visualNum a brick-ev:EV_Charging_Session_Visual_Number .
?visualNum  ref:hasExternalReference ?visualNumExt .
?visualNumExt ref-ext:hasRealtimeTimeseriesId ?visualNumId .

}
    """

df = do_config(query)
df

""


# Configuration

In [223]:
def sensor_metadata_dynamic(dp_class, timeseries,variable_name):
    query = f"""
    PREFIX ref-ext: <https://w3id.org/b4b/schemes/ext/ref-ext#>
    PREFIX ref: <https://brickschema.org/schema/Brick/ref#>
    PREFIX brick: <https://brickschema.org/schema/Brick#>
    PREFIX sba: <https://w3id.org/b4b/schemes/algorithms/sba#> 
    PREFIX mpc: <https://w3id.org/b4b/schemes/algorithms/mpc#> 
    PREFIX forc: <https://w3id.org/b4b/schemes/algorithms/forc#> 

    SELECT DISTINCT ?{variable_name} ?Tsid ?DataSource ?APIPath ?refType 
    WHERE {{ 
    ?cls a  <{dp_class}> .
    ?cls  ref:hasExternalReference ?Ext .
    ?Ext a ?refType .
    ?Ext ref-ext:has{timeseries}TimeseriesId ?Tsid .
    ?Ext ref:storedAt ?DataSource .
    ?DataSource ref-ext:connstring ?APIPath .
    BIND(<{dp_class}> AS ?{variable_name}) .
    }}
    """

    df = do_config(query)
    # df = do_prefixes(df)
    nested_json = {}
    for index, row in df.iterrows():
        key = df.columns[0]
        if key not in nested_json:
            nested_json[key] = {
                'Point':dp_class,
                f'{df.columns[0]}{df.columns[1]}': row[df.columns[1]],
                f'{df.columns[0]}{df.columns[2]}': row[df.columns[2]],
                f'{df.columns[0]}{df.columns[3]}': row[df.columns[3]],
                f'{df.columns[0]}{df.columns[4]}': row[df.columns[4]],
            }
    # print(json.dumps(nested_json, indent=4))
    return nested_json

In [224]:
def sensor_metadata_static(dp_class, variable_name):
    query = f"""
    PREFIX ref-ext: <https://w3id.org/b4b/schemes/ext/ref-ext#>
    PREFIX ref: <https://brickschema.org/schema/Brick/ref#>
    PREFIX brick: <https://brickschema.org/schema/Brick#>
    PREFIX sba: <https://w3id.org/b4b/schemes/algorithms/sba#> 
    PREFIX mpc: <https://w3id.org/b4b/schemes/algorithms/mpc#> 
    PREFIX forc: <https://w3id.org/b4b/schemes/algorithms/forc#> 

    SELECT DISTINCT ?{variable_name} ?Val ?Unit 
    WHERE {{ 
    ?cls a  <{dp_class}> .
    ?cls  brick:value ?Val .
    ?cls brick:hasUnit ?Unit .
    BIND(<{dp_class}> AS ?{variable_name}) .
    }}
    """

    df = do_config(query)
    # df = do_prefixes(df)
    nested_json = {}
    for index, row in df.iterrows():
        key = df.columns[0]
        if key not in nested_json:
            nested_json[key] = {
                'Point':dp_class,
                f'{df.columns[0]}{df.columns[1]}': row[df.columns[1]],
                f'{df.columns[0]}{df.columns[2]}': row[df.columns[2]],
            }
    # print(json.dumps(nested_json, indent=4))
    return nested_json

In [ ]:
def gen_configs(inputs_arr, config_type):
    all_configs={}
    data_type=['Historical', 'Realtime', 'Forecast', 'LastKnown']

    if config_type == 'dynamic':
        for input in inputs_arr:
            sensor_class = input.split('#')[1]
            variable_name = input.split('#')[1].replace("_", "")
            for data in data_type:
                res = sensor_metadata_dynamic(input, data, variable_name)
                all_configs.update(res)
    elif config_type == 'static':
        for input in inputs_arr:
            sensor_class = input.split('#')[1]
            variable_name = input.split('#')[1].replace("_", "")
            res = sensor_metadata_static(input, variable_name)
            all_configs.update(res)
    # print(json.dumps(all_configs, indent=4))
    return all_configs

### Load forecast

In [248]:
res=gen_configs(forc_inputs, config_type='dynamic')
res 

{'BuildingElectricalMeter': {'Point': 'https://brickschema.org/schema/Brick#Building_Electrical_Meter',
  'BuildingElectricalMeterTsid': 'f30ca6c6d2b3111dee9066951e270141',
  'BuildingElectricalMeterDataSource': 'urn:building_v1#insitereports',
  'BuildingElectricalMeterAPIPath': 'https://ivs.kropman.nl/insitereports/',
  'BuildingElectricalMeterrefType': 'https://w3id.org/b4b/schemes/ext/ref-ext#HistoricalTimeseriesReference'},
 'SolarRadianceSensor': {'Point': 'https://brickschema.org/schema/Brick#Solar_Radiance_Sensor',
  'SolarRadianceSensorTsid': 'a683b494a806af9a8da4d35ae99ba763',
  'SolarRadianceSensorDataSource': 'urn:building_v1#insitereports',
  'SolarRadianceSensorAPIPath': 'https://ivs.kropman.nl/insitereports/',
  'SolarRadianceSensorrefType': 'https://w3id.org/b4b/schemes/ext/ref-ext#HistoricalTimeseriesReference'},
 'OutdoorTemperatureForecast': {'Point': 'https://w3id.org/b4b/schemes/algorithms/forc#OutdoorTemperatureForecast',
  'OutdoorTemperatureForecastTsid': '3ede3

In [227]:
def extract_configs(res, config_var='Tsid'):
    extracted_configs = {}
    for item in res.values():
        for key, value in item.items():
            if config_var in key:
                # print(f"{key}={value}")
                extracted_configs[key] = value
    return extracted_configs

In [249]:
extract_configs(res, config_var='Tsid')

{'BuildingElectricalMeterTsid': 'f30ca6c6d2b3111dee9066951e270141',
 'SolarRadianceSensorTsid': 'a683b494a806af9a8da4d35ae99ba763',
 'OutdoorTemperatureForecastTsid': '3ede3817d96d7bb87865eaa7ac657860',
 'WindSpeedForecastTsid': '3ede3817d96d7bb87865eaa7ac657860',
 'SolarRadianceForecastTsid': '3ede3817d96d7bb87865eaa7ac657860'}

In [229]:
extract_configs(res, config_var='Source')

{'BuildingElectricalMeterDataSource': 'urn:building_v1#insitereports',
 'SolarRadianceSensorDataSource': 'urn:building_v1#insitereports',
 'OutdoorTemperatureForecastDataSource': 'urn:building_v1#insitereports',
 'WindSpeedForecastDataSource': 'urn:building_v1#insitereports',
 'SolarRadianceForecastDataSource': 'urn:building_v1#insitereports'}

In [230]:
extract_configs(res, config_var='API')

{'BuildingElectricalMeterAPIPath': 'https://ivs.kropman.nl/insitereports/',
 'SolarRadianceSensorAPIPath': 'https://ivs.kropman.nl/insitereports/',
 'OutdoorTemperatureForecastAPIPath': 'https://ivs.kropman.nl/insitereports/',
 'WindSpeedForecastAPIPath': 'https://ivs.kropman.nl/insitereports/',
 'SolarRadianceForecastAPIPath': 'https://ivs.kropman.nl/insitereports/'}

In [231]:
extract_configs(res, config_var='ref')

{'BuildingElectricalMeterrefType': 'https://w3id.org/b4b/schemes/ext/ref-ext#HistoricalTimeseriesReference',
 'SolarRadianceSensorrefType': 'https://w3id.org/b4b/schemes/ext/ref-ext#HistoricalTimeseriesReference',
 'OutdoorTemperatureForecastrefType': 'https://w3id.org/b4b/schemes/ext/ref-ext#ForecastTimeseriesReference',
 'WindSpeedForecastrefType': 'https://w3id.org/b4b/schemes/ext/ref-ext#ForecastTimeseriesReference',
 'SolarRadianceForecastrefType': 'https://w3id.org/b4b/schemes/ext/ref-ext#ForecastTimeseriesReference'}

## Realtime feedback

In [232]:
gen_configs(feedback_inputs, config_type='dynamic')

{'EVArrivalTime': {'Point': 'https://w3id.org/b4b/schemes/ext/brick-ev#EV_Arrival_Time',
  'EVArrivalTimeTsid': 'evse/welkomkropman',
  'EVArrivalTimeDataSource': 'urn:building_v1#welkomkropman',
  'EVArrivalTimeAPIPath': 'https://ivs.kropman.nl/wk/',
  'EVArrivalTimerefType': 'https://w3id.org/b4b/schemes/ext/ref-ext#RealtimeTimeseriesReference'},
 'EVArrivalSOC': {'Point': 'https://w3id.org/b4b/schemes/ext/brick-ev#EV_Arrival_SOC',
  'EVArrivalSOCTsid': 'evse/welkomkropman',
  'EVArrivalSOCDataSource': 'urn:building_v1#welkomkropman',
  'EVArrivalSOCAPIPath': 'https://ivs.kropman.nl/wk/',
  'EVArrivalSOCrefType': 'https://w3id.org/b4b/schemes/ext/ref-ext#RealtimeTimeseriesReference'},
 'EVBatterySize': {'Point': 'https://w3id.org/b4b/schemes/ext/brick-ev#EV_Battery_Size',
  'EVBatterySizeTsid': 'evse/welkomkropman',
  'EVBatterySizeDataSource': 'urn:building_v1#welkomkropman',
  'EVBatterySizeAPIPath': 'https://ivs.kropman.nl/wk/',
  'EVBatterySizerefType': 'https://w3id.org/b4b/sche

## Static params

In [233]:
res=gen_configs(static_param_inputs, config_type='static')
res

{'ChargerVoltageRating': {'Point': 'https://w3id.org/b4b/schemes/ext/brick-ev#Charger_Voltage_Rating',
  'ChargerVoltageRatingVal': '230',
  'ChargerVoltageRatingUnit': 'http://qudt.org/vocab/unit/Volt'},
 'ChargerPowerRating': {'Point': 'https://w3id.org/b4b/schemes/ext/brick-ev#Charger_Power_Rating',
  'ChargerPowerRatingVal': '11',
  'ChargerPowerRatingUnit': 'http://qudt.org/vocab/unit/KiloW'},
 'PowerFactor': {'Point': 'https://w3id.org/b4b/schemes/ext/brick-ext#Power_Factor',
  'PowerFactorVal': '0.95',
  'PowerFactorUnit': 'http://qudt.org/vocab/unit/Unitless'}}

In [234]:
extract_configs(res, config_var='Val') 

{'ChargerVoltageRatingVal': '230',
 'ChargerPowerRatingVal': '11',
 'PowerFactorVal': '0.95'}

In [235]:
extract_configs(res, config_var='Unit') 

{'ChargerVoltageRatingUnit': 'http://qudt.org/vocab/unit/Volt',
 'ChargerPowerRatingUnit': 'http://qudt.org/vocab/unit/KiloW',
 'PowerFactorUnit': 'http://qudt.org/vocab/unit/Unitless'}

## Dynamic params

In [236]:
res=gen_configs(dynamic_param_inputs, config_type='dynamic')
res

{'EVElectricEnergyMeter': {'Point': 'https://w3id.org/b4b/schemes/ext/brick-ev#EV_Electric_Energy_Meter',
  'EVElectricEnergyMeterTsid': '759ca9cc4f3bf8f552084d67f25d45899',
  'EVElectricEnergyMeterDataSource': 'urn:building_v1#insiteview',
  'EVElectricEnergyMeterAPIPath': 'https://ivs.kropman.nl/insiteview/',
  'EVElectricEnergyMeterrefType': 'https://w3id.org/b4b/schemes/ext/ref-ext#HistoricalTimeseriesReference'},
 'EVElectricPowerSensor': {'Point': 'https://w3id.org/b4b/schemes/ext/brick-ev#EV_Electric_Power_Sensor',
  'EVElectricPowerSensorTsid': '56b56a995eae7ac5e331e115b3ad9a708.AVERAGE_POWER$value',
  'EVElectricPowerSensorDataSource': 'urn:building_v1#insiteview',
  'EVElectricPowerSensorAPIPath': 'https://ivs.kropman.nl/insiteview/',
  'EVElectricPowerSensorrefType': 'https://w3id.org/b4b/schemes/ext/ref-ext#HistoricalTimeseriesReference'}}

In [237]:
extract_configs(res, config_var='Tsid')

{'EVElectricEnergyMeterTsid': '759ca9cc4f3bf8f552084d67f25d45899',
 'EVElectricPowerSensorTsid': '56b56a995eae7ac5e331e115b3ad9a708.AVERAGE_POWER$value'}

In [238]:
extract_configs(res, config_var='Source')

{'EVElectricEnergyMeterDataSource': 'urn:building_v1#insiteview',
 'EVElectricPowerSensorDataSource': 'urn:building_v1#insiteview'}

In [239]:
extract_configs(res, config_var='API')

{'EVElectricEnergyMeterAPIPath': 'https://ivs.kropman.nl/insiteview/',
 'EVElectricPowerSensorAPIPath': 'https://ivs.kropman.nl/insiteview/'}

## Constraints

In [240]:
res=gen_configs(constraints_inputs, config_type='static')
res

{'MinimumChargingCurrentSetpoint': {'Point': 'https://w3id.org/b4b/schemes/ext/brick-ev#Minimum_Charging_Current_Setpoint',
  'MinimumChargingCurrentSetpointVal': '6',
  'MinimumChargingCurrentSetpointUnit': 'http://qudt.org/vocab/unit/A'},
 'MaximumChargingCurrentSetpoint': {'Point': 'https://w3id.org/b4b/schemes/ext/brick-ev#Maximum_Charging_Current_Setpoint',
  'MaximumChargingCurrentSetpointVal': '16',
  'MaximumChargingCurrentSetpointUnit': 'http://qudt.org/vocab/unit/A'}}

In [241]:
extract_configs(res, config_var='Val') 

{'MinimumChargingCurrentSetpointVal': '6',
 'MaximumChargingCurrentSetpointVal': '16'}

In [242]:
extract_configs(res, config_var='Unit') 

{'MinimumChargingCurrentSetpointUnit': 'http://qudt.org/vocab/unit/A',
 'MaximumChargingCurrentSetpointUnit': 'http://qudt.org/vocab/unit/A'}

## States

In [243]:
res=gen_configs(state_inputs, config_type='dynamic')
res

{'EVArrivalTime': {'Point': 'https://w3id.org/b4b/schemes/ext/brick-ev#EV_Arrival_Time',
  'EVArrivalTimeTsid': 'evse/welkomkropman',
  'EVArrivalTimeDataSource': 'urn:building_v1#welkomkropman',
  'EVArrivalTimeAPIPath': 'https://ivs.kropman.nl/wk/',
  'EVArrivalTimerefType': 'https://w3id.org/b4b/schemes/ext/ref-ext#RealtimeTimeseriesReference'},
 'EVArrivalSOC': {'Point': 'https://w3id.org/b4b/schemes/ext/brick-ev#EV_Arrival_SOC',
  'EVArrivalSOCTsid': 'evse/welkomkropman',
  'EVArrivalSOCDataSource': 'urn:building_v1#welkomkropman',
  'EVArrivalSOCAPIPath': 'https://ivs.kropman.nl/wk/',
  'EVArrivalSOCrefType': 'https://w3id.org/b4b/schemes/ext/ref-ext#RealtimeTimeseriesReference'},
 'EVBatterySize': {'Point': 'https://w3id.org/b4b/schemes/ext/brick-ev#EV_Battery_Size',
  'EVBatterySizeTsid': 'evse/welkomkropman',
  'EVBatterySizeDataSource': 'urn:building_v1#welkomkropman',
  'EVBatterySizeAPIPath': 'https://ivs.kropman.nl/wk/',
  'EVBatterySizerefType': 'https://w3id.org/b4b/sche

In [244]:
extract_configs(res, config_var='Tsid')

{'EVArrivalTimeTsid': 'evse/welkomkropman',
 'EVArrivalSOCTsid': 'evse/welkomkropman',
 'EVBatterySizeTsid': 'evse/welkomkropman',
 'VehicleFullStatusTsid': '759ca9cc4f3bf8f552084d67f25d4589',
 'ChargingSessionStatusTsid': '759ca9cc4f3bf8f552084d67f25d4589',
 'EVElectricEnergyMeterTsid': '759ca9cc4f3bf8f552084d67f25d45899',
 'EVChargingSessionVisualNumberTsid': '759ca9cc4f3bf8f552084d67f25d4589',
 'EVLoadCurrentSensorTsid': 'cd8afb76be06cc43e288041260df100eb.CURRENT_L1$value'}

In [245]:
extract_configs(res, config_var='API')

{'EVArrivalTimeAPIPath': 'https://ivs.kropman.nl/wk/',
 'EVArrivalSOCAPIPath': 'https://ivs.kropman.nl/wk/',
 'EVBatterySizeAPIPath': 'https://ivs.kropman.nl/wk/',
 'VehicleFullStatusAPIPath': 'https://ivs.kropman.nl/insiteview/',
 'ChargingSessionStatusAPIPath': 'https://ivs.kropman.nl/insiteview/',
 'EVElectricEnergyMeterAPIPath': 'https://ivs.kropman.nl/insiteview/',
 'EVChargingSessionVisualNumberAPIPath': 'https://ivs.kropman.nl/insiteview/',
 'EVLoadCurrentSensorAPIPath': 'https://ivs.kropman.nl/insiteview/'}

In [246]:
extract_configs(res, config_var='Source')

{'EVArrivalTimeDataSource': 'urn:building_v1#welkomkropman',
 'EVArrivalSOCDataSource': 'urn:building_v1#welkomkropman',
 'EVBatterySizeDataSource': 'urn:building_v1#welkomkropman',
 'VehicleFullStatusDataSource': 'urn:building_v1#insiteview',
 'ChargingSessionStatusDataSource': 'urn:building_v1#insiteview',
 'EVElectricEnergyMeterDataSource': 'urn:building_v1#insiteview',
 'EVChargingSessionVisualNumberDataSource': 'urn:building_v1#insiteview',
 'EVLoadCurrentSensorDataSource': 'urn:building_v1#insiteview'}

# Shapes

In [304]:
def generate_dynamic_shape_graphs(sparql_endpoint, sparql_query, folder_name):
    import uuid

    query_params = {
        'query': sparql_query,
        'infer': 'false' 
    }
    headers = {
    "Content-Type": "application/sparql-query",
    "Accept": "application/json"
    }
    auth = ("lchamari", "lchamari")

    response = requests.get(sparql_endpoint, params=query_params, headers=headers, auth=auth)
    
    prefixes = {
        'brick:': 'https://brickschema.org/schema/Brick#',
        'ref:': 'https://brickschema.org/schema/Brick/ref#',
        'ref-ext:': 'https://w3id.org/b4b/schemes/ext/ref-ext#',
        'sba:': 'https://w3id.org/b4b/schemes/algorithms/sba#', 
        'mpc:': 'https://w3id.org/b4b/schemes/algorithms/mpc#', 
        'forc:': 'https://w3id.org/b4b/schemes/algorithms/forc#', 
        'brick-ev:': '<https://w3id.org/b4b/schemes/ext/brick-ev#>'
    }

    response_json = json.loads(response.text)
    bindings = response_json["results"]["bindings"]
    data = [
        {"Input": item["input"]["value"], "Timeseries Reference": item["tsref"]["value"]}
        for item in bindings
    ]
    df = pd.DataFrame(data)

    class_list = df['Input'].tolist()

    point_with_suffix_list = []
    for point in class_list:
        suffix = point.split('#')[0] + '#'
        point = point.split('#')[-1]

        for prefix, uri in prefixes.items():
            if uri == suffix:
                suffix = prefix
                point_with_suffix = f'{suffix}:{point}'
                point_with_suffix_list.append(point_with_suffix)
                break
  
    dplist = {}
    for index, row in df.iterrows():
        if row['Input'] and row['Timeseries Reference']:
            dplist[row['Input']] = row['Timeseries Reference']
    # print(dplist)

    for key, value in dplist.items():
        for prefix, uri in prefixes.items():
            if uri in value:
                dplist[key] = value.replace(uri, prefix)
    dplist = {key.replace('https://w3id.org/b4b/schemes/algorithms/forc#', 'forc:'): value for key, value in dplist.items()}
    dplist = {key.replace('https://brickschema.org/schema/Brick#', 'brick:'): value for key, value in dplist.items()}
    dplist = {key.replace('https://w3id.org/b4b/schemes/algorithms/sba#', 'sba:'): value for key, value in dplist.items()}
    dplist = {key.replace('https://w3id.org/b4b/schemes/algorithms/mpc#', 'mpc:'): value for key, value in dplist.items()}
    dplist = {key.replace('https://brickschema.org/schema/Brick/ref#', 'ref:'): value for key, value in dplist.items()}
    dplist = {key.replace('https://w3id.org/b4b/schemes/ext/ref-ext#', 'ref-ext:'): value for key, value in dplist.items()}
    dplist = {key.replace('https://w3id.org/b4b/schemes/ext/brick-ev#', 'brick-ev:'): value for key, value in dplist.items()}

    dict = {
    "ref-ext:LastKnownValueTimeseriesReference": "ref-ext:hasLastKnownValueTimeseriesId",
    "ref-ext:HistoricalTimeseriesReference": "ref-ext:hasHistoricalTimeseriesId",
    "ref-ext:RealtimeTimeseriesReference": "ref-ext:hasRealtimeTimeseriesId",
    "ref-ext:ForecastTimeseriesReference": "ref-ext:hasForecastTimeseriesId",
    "ref:TimeseriesReference": "ref:hasTimeseriesId",
    }

    shape_graph_list = []
    class_names = []
    class_names_ = []
    for k in dplist:
        value_2 = dict[dplist[k]]
        shape_graph = f"""
@prefix brick: <https://brickschema.org/schema/Brick#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix sh: <http://www.w3.org/ns/shacl#>.
@prefix ref: <https://brickschema.org/schema/Brick/ref#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix ref-ext: <https://w3id.org/b4b/schemes/ext/ref-ext#> .
@prefix sba: <https://w3id.org/b4b/schemes/algorithms/sba#> .
@prefix mpc: <https://w3id.org/b4b/schemes/algorithms/mpc#> .
@prefix forc: <https://w3id.org/b4b/schemes/algorithms/forc#> .
@prefix brick-ev: <https://w3id.org/b4b/schemes/ext/brick-ev#> .
@prefix : <urn:algorithm_shapes#> .

:requiredClass_{uuid.uuid4()}
    a sh:NodeShape ;
    sh:targetNode :targetNode ;
    sh:sparql [
        a sh:SPARQLConstraint ;
        sh:message "The building must contain at least one instance of the data point." ;
        sh:select \"\"\"
            SELECT ?this
            WHERE {{
                FILTER NOT EXISTS {{
                    ?sensor a {k}  .
                }}
            }}
        \"\"\" ;
    ] .
:requiedRelationship_{uuid.uuid4()}
    a sh:NodeShape ;
    sh:targetClass {k} ;
    sh:property [
        sh:path ref:hasExternalReference;
        sh:minCount 1 ;
        sh:message "All datapoints need a timeseries reference." ;
    ] .
:reqioredExrefValue_{uuid.uuid4()}
    a sh:NodeShape ;
    sh:targetClass {dplist[k]} ; 
    sh:property [
        sh:path {value_2};
        sh:datatype xsd:string ;    
        sh:minCount 1 ;
        sh:maxCount 1 ;
        sh:message "All timeseries references need a value." ;
    ] .

"""
        shape_graph_list.append(shape_graph)
        class_names.append(k)
        class_name = k.replace(":", "_")
        class_names_.append(class_name)
        
    i=0
    for g in shape_graph_list:
        with open(f"shacl/{folder_name}/shape_graph_{class_names_[i]}.ttl", "w") as f:
            f.write(g)
            i=i+1

    return df

In [305]:
sparql_endpoint = 'https://graphdb.bimsim.app/repositories/breda_plus_schemas'

### LoadForecastAlgorithm

In [306]:
query = """
PREFIX ref-ext: <https://w3id.org/b4b/schemes/ext/ref-ext#>
PREFIX ref: <https://brickschema.org/schema/Brick/ref#>
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX sba: <https://w3id.org/b4b/schemes/algorithms/sba#> 
PREFIX mpc: <https://w3id.org/b4b/schemes/algorithms/mpc#> 
PREFIX forc: <https://w3id.org/b4b/schemes/algorithms/forc#> 

select ?input ?tsref where {
    ?s ?p ?input .
    FILTER(?s=sba:LoadForecastAlgorithm)
    FILTER(?p=sba:hasInputData)
    ?input ref:hasExternalReference ?arr .
    ?arr a ?tsref .
}
    """
generate_dynamic_shape_graphs(sparql_endpoint, query, folder_name="/dynamic/LoadForecastAlgorithm")


,Input,Timeseries Reference
0,https://brickschema.org/schema/Brick#Building_...,https://w3id.org/b4b/schemes/ext/ref-ext#Histo...
1,https://brickschema.org/schema/Brick#Outside_A...,https://w3id.org/b4b/schemes/ext/ref-ext#Histo...
2,https://brickschema.org/schema/Brick#Wind_Spee...,https://w3id.org/b4b/schemes/ext/ref-ext#Histo...
3,https://brickschema.org/schema/Brick#Solar_Rad...,https://w3id.org/b4b/schemes/ext/ref-ext#Histo...
4,https://w3id.org/b4b/schemes/algorithms/forc#O...,https://w3id.org/b4b/schemes/ext/ref-ext#Forec...
5,https://w3id.org/b4b/schemes/algorithms/forc#W...,https://w3id.org/b4b/schemes/ext/ref-ext#Forec...
6,https://w3id.org/b4b/schemes/algorithms/forc#S...,https://w3id.org/b4b/schemes/ext/ref-ext#Forec...


### EVRealtimeFeedbackAlgorithm


In [307]:
query = """
PREFIX ref-ext: <https://w3id.org/b4b/schemes/ext/ref-ext#>
PREFIX ref: <https://brickschema.org/schema/Brick/ref#>
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX sba: <https://w3id.org/b4b/schemes/algorithms/sba#> 
PREFIX mpc: <https://w3id.org/b4b/schemes/algorithms/mpc#> 
PREFIX forc: <https://w3id.org/b4b/schemes/algorithms/forc#> 

select ?input ?tsref where {
    ?s ?p ?input .
    FILTER(?s=mpc:EVRealtimeFeedbackAlgorithm)
    FILTER(?p=sba:hasInputData)
    ?input ref:hasExternalReference ?arr .
    ?arr a ?tsref .
}
    """

generate_dynamic_shape_graphs(sparql_endpoint, query, folder_name='/dynamic/RealtimeFeedbackAlgorithm')

,Input,Timeseries Reference
0,https://w3id.org/b4b/schemes/ext/brick-ev#EV_C...,https://w3id.org/b4b/schemes/ext/ref-ext#Realt...
1,https://w3id.org/b4b/schemes/ext/brick-ev#EV_A...,https://w3id.org/b4b/schemes/ext/ref-ext#Realt...
2,https://w3id.org/b4b/schemes/ext/brick-ev#EV_D...,https://w3id.org/b4b/schemes/ext/ref-ext#Realt...
3,https://w3id.org/b4b/schemes/ext/brick-ev#EV_A...,https://w3id.org/b4b/schemes/ext/ref-ext#Realt...
4,https://w3id.org/b4b/schemes/ext/brick-ev#EV_B...,https://w3id.org/b4b/schemes/ext/ref-ext#Realt...


### EVStateEstimationAlgorithm

In [308]:
query = """
PREFIX ref-ext: <https://w3id.org/b4b/schemes/ext/ref-ext#>
PREFIX ref: <https://brickschema.org/schema/Brick/ref#>
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX sba: <https://w3id.org/b4b/schemes/algorithms/sba#> 
PREFIX mpc: <https://w3id.org/b4b/schemes/algorithms/mpc#> 
PREFIX forc: <https://w3id.org/b4b/schemes/algorithms/forc#> 

select ?input ?tsref where {
    ?s ?p ?input .
    FILTER(?s=mpc:EVStateEstimationAlgorithm)
    FILTER(?p=sba:hasInputData)
    ?input ref:hasExternalReference ?arr .
    ?arr a ?tsref .
}
    """

generate_dynamic_shape_graphs(sparql_endpoint, query, folder_name='dynamic/EVStateEstimationAlgorithm')

,Input,Timeseries Reference
0,https://w3id.org/b4b/schemes/ext/brick-ev#EV_C...,https://w3id.org/b4b/schemes/ext/ref-ext#Realt...
1,https://w3id.org/b4b/schemes/ext/brick-ev#EV_A...,https://w3id.org/b4b/schemes/ext/ref-ext#Realt...
2,https://w3id.org/b4b/schemes/ext/brick-ev#EV_D...,https://w3id.org/b4b/schemes/ext/ref-ext#Realt...
3,https://w3id.org/b4b/schemes/ext/brick-ev#EV_A...,https://w3id.org/b4b/schemes/ext/ref-ext#Realt...
4,https://w3id.org/b4b/schemes/ext/brick-ev#EV_B...,https://w3id.org/b4b/schemes/ext/ref-ext#Realt...
5,https://w3id.org/b4b/schemes/ext/brick-ev#Vehi...,https://w3id.org/b4b/schemes/ext/ref-ext#Realt...
6,https://w3id.org/b4b/schemes/ext/brick-ev#Char...,https://w3id.org/b4b/schemes/ext/ref-ext#Realt...
7,https://w3id.org/b4b/schemes/ext/brick-ev#EV_E...,https://w3id.org/b4b/schemes/ext/ref-ext#Realt...
8,https://w3id.org/b4b/schemes/ext/brick-ev#EV_C...,https://w3id.org/b4b/schemes/ext/ref-ext#Realt...


### DynamicParameterEstimationAlgorithm

In [309]:
query = """
PREFIX ref-ext: <https://w3id.org/b4b/schemes/ext/ref-ext#>
PREFIX ref: <https://brickschema.org/schema/Brick/ref#>
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX sba: <https://w3id.org/b4b/schemes/algorithms/sba#> 
PREFIX mpc: <https://w3id.org/b4b/schemes/algorithms/mpc#> 
PREFIX forc: <https://w3id.org/b4b/schemes/algorithms/forc#> 

select ?input ?tsref where {
    ?s ?p ?input .
    FILTER(?s=mpc:DynamicParameterEstimationAlgorithm)
    FILTER(?p=sba:hasInputData)
    ?input ref:hasExternalReference ?arr .
    ?arr a ?tsref .
}
    """

generate_dynamic_shape_graphs(sparql_endpoint, query, folder_name='/dynamic/DynamicParameterEstimationAlgorithm')

,Input,Timeseries Reference
0,https://w3id.org/b4b/schemes/ext/brick-ev#EV_E...,https://w3id.org/b4b/schemes/ext/ref-ext#Realt...


### EVChargingCurrentSetpointCommunication

In [310]:
query = """
PREFIX ref-ext: <https://w3id.org/b4b/schemes/ext/ref-ext#>
PREFIX ref: <https://brickschema.org/schema/Brick/ref#>
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX sba: <https://w3id.org/b4b/schemes/algorithms/sba#> 
PREFIX mpc: <https://w3id.org/b4b/schemes/algorithms/mpc#> 
PREFIX forc: <https://w3id.org/b4b/schemes/algorithms/forc#> 

select ?input ?tsref where {
    ?s ?p ?input .
    FILTER(?s=mpc:EVChargingCurrentSetpointCommunication)
    FILTER(?p=sba:hasInputData)
    ?input ref:hasExternalReference ?arr .
    ?arr a ?tsref .
}
    """

df = generate_dynamic_shape_graphs(sparql_endpoint, query, folder_name='/dynamic/EVChargingCurrentSetpointCommunication')

KeyError: 'Input'

# Validate

## Breda

Points under Dynamic configurations --> have a reference to timeseries \\\
Points under Static configurations --> have a brick:value

In [23]:
building_sparql_endpoint = 'https://graphdb.bimsim.app/repositories/breda'

In [24]:
from pyshacl import validate
from rdflib import Graph

def validate_graph(data_graph, shape_graph):
    conforms, report_graph, report_text = validate(
        data_graph,
        shacl_graph=shape_graph,
        inference='none',
        serialize_report_graph=True
    )
    return conforms, report_graph, report_text



In [25]:
construct_query = """
CONSTRUCT { ?s ?p ?o }
WHERE { ?s ?p ?o }
"""
query_params = {
    'query': construct_query,
    'infer': 'true' 
}
headers = {
"Content-Type": "application/sparql-query",
"Accept": "application/json"
}
auth = ("lchamari", "lchamari")

response = requests.get(building_sparql_endpoint, params=query_params,  auth=auth)

if response.status_code == 200:
    response_text = response.text
    building_data_graph= Graph()
    building_data_graph.parse(data=response_text, format="turtle")
    print(f"Loaded {len(building_data_graph)} tripples")

else:

    print(f"Error: {response.status_code} - {response.reason}")

Loaded 774 tripples


In [26]:
shape_graph= Graph()
shape_graph.parse("shacl/dynamic/LoadForecastAlgorithm/shape_graph_brick_Building_Electrical_Meter.ttl", format="turtle")
conforms, report_graph, report_text  = validate_graph(building_data_graph, shape_graph)
print("Conforms:", conforms)
print(report_text)

Conforms: True
Validation Report
Conforms: True



In [27]:
def validate_algorithm(folder_name):
    full_text = ""
    folder_path = f"shacl\{folder_name}"
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.ttl'):
            file_path = os.path.join(folder_path, file_name)
            temp_graph = Graph()
            temp_graph.parse(file_path, format='turtle')
            res = validate_graph(building_data_graph, temp_graph)
            conforms, results_graph, results_text = res
            full_text = full_text + file_name + "\n"
            full_text = full_text + results_text + "\n"
    try:
        folder_path = f"validation/building_1/{folder_name}"
        os.makedirs(folder_path, exist_ok=True)
        with open(f"{folder_path}/validation.txt", "w") as f:
            f.write(str(full_text))
        print("Writing to:", os.path.abspath(f"{folder_path}/validation.txt"))
        return full_text
    except Exception as e:
        print("Error writing file:", e)
        return None


In [28]:
shape_graph.bind('forc', 'https://w3id.org/b4b/schemes/algorithms/forc#')
building_data_graph.bind('forc', 'https://w3id.org/b4b/schemes/algorithms/forc#')
res=validate_algorithm('dynamic/LoadForecastAlgorithm')
print(res)

Writing to: c:\Users\lasit\Documents\SoftwarePhD\mpc-schema-journal-paper\validation\building_1\dynamic\LoadForecastAlgorithm\validation.txt
shape_graph_brick_Building_Electrical_Meter.ttl
Validation Report
Conforms: True

shape_graph_brick_Outside_Air_Temperature_Sensor.ttl
Validation Report
Conforms: True

shape_graph_brick_Solar_Radiance_Sensor.ttl
Validation Report
Conforms: True

shape_graph_brick_Wind_Speed_Sensor.ttl
Validation Report
Conforms: True

shape_graph_forc_OutdoorTemperatureForecast.ttl
Validation Report
Conforms: True

shape_graph_forc_SolarRadianceForecast.ttl
Validation Report
Conforms: True

shape_graph_forc_WindSpeedForecast.ttl
Validation Report
Conforms: True




In [29]:
shape_graph.bind('brick-ev', 'https://w3id.org/b4b/schemes/ext/brick-ev#')
building_data_graph.bind('brick-ev', 'https://w3id.org/b4b/schemes/ext/brick-ev#')
res=validate_algorithm('dynamic/EVStateEstimationAlgorithm')
print(res)

Writing to: c:\Users\lasit\Documents\SoftwarePhD\mpc-schema-journal-paper\validation\building_1\dynamic\EVStateEstimationAlgorithm\validation.txt
shape_graph_brick-ev_Charging_Session_Status.ttl
Validation Report
Conforms: True

shape_graph_brick-ev_EV_Arrival_SOC.ttl
Validation Report
Conforms: True

shape_graph_brick-ev_EV_Arrival_Time.ttl
Validation Report
Conforms: True

shape_graph_brick-ev_EV_Battery_Size.ttl
Validation Report
Conforms: True

shape_graph_brick-ev_EV_Charging_Session_Visual_Number.ttl
Validation Report
Conforms: True

shape_graph_brick-ev_EV_Charging_Station_ID.ttl
Validation Report
Conforms: True

shape_graph_brick-ev_EV_Departure_Time.ttl
Validation Report
Conforms: True

shape_graph_brick-ev_EV_Electric_Energy_Meter.ttl
Validation Report
Conforms: True

shape_graph_brick-ev_Vehicle_Full_Status.ttl
Validation Report
Conforms: True




In [30]:
shape_graph.bind('brick-ev', 'https://w3id.org/b4b/schemes/ext/brick-ev#')
building_data_graph.bind('brick-ev', 'https://w3id.org/b4b/schemes/ext/brick-ev#')
res=validate_algorithm('dynamic/RealtimeFeedbackAlgorithm')
print(res)

Writing to: c:\Users\lasit\Documents\SoftwarePhD\mpc-schema-journal-paper\validation\building_1\dynamic\RealtimeFeedbackAlgorithm\validation.txt
shape_graph_brick-ev_EV_Arrival_SOC.ttl
Validation Report
Conforms: True

shape_graph_brick-ev_EV_Arrival_Time.ttl
Validation Report
Conforms: True

shape_graph_brick-ev_EV_Battery_Size.ttl
Validation Report
Conforms: True

shape_graph_brick-ev_EV_Charging_Station_ID.ttl
Validation Report
Conforms: True

shape_graph_brick-ev_EV_Departure_Time.ttl
Validation Report
Conforms: True




In [31]:
res=validate_algorithm('dynamic/EVChargingCurrentSetpointCommunication')
print(res)

Writing to: c:\Users\lasit\Documents\SoftwarePhD\mpc-schema-journal-paper\validation\building_1\dynamic\EVChargingCurrentSetpointCommunication\validation.txt
shape_graph_brick-ev_EV_Electric_Energy_Meter.ttl
Validation Report
Conforms: True




## Building_v1

In [32]:
building_sparql_endpoint = 'https://graphdb.bimsim.app/repositories/building_v1'

In [33]:
construct_query = """
CONSTRUCT { ?s ?p ?o }
WHERE { ?s ?p ?o }
"""
query_params = {
    'query': construct_query,
    'infer': 'true' 
}
headers = {
"Content-Type": "application/sparql-query",
"Accept": "application/json"
}
auth = ("lchamari", "lchamari")

response = requests.get(building_sparql_endpoint, params=query_params,  auth=auth)

if response.status_code == 200:
    response_text = response.text
    building_data_graph= Graph()
    building_data_graph.parse(data=response_text, format="turtle")
    print(f"Loaded {len(building_data_graph)} tripples")

else:
    print(f"Error: {response.status_code} - {response.reason}")

Loaded 719 tripples


In [34]:
def validate_algorithm(folder_name):
    full_text = ""
    folder_path = f"shacl\{folder_name}"
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.ttl'):
            file_path = os.path.join(folder_path, file_name)
            temp_graph = Graph()
            temp_graph.parse(file_path, format='turtle')
            res = validate_graph(building_data_graph, temp_graph)
            conforms, results_graph, results_text = res
            full_text = full_text + file_name + "\n"
            full_text = full_text + results_text + "\n"
    try:
        folder_path = f"validation/building_v1/{folder_name}"
        os.makedirs(folder_path, exist_ok=True)
        with open(f"{folder_path}/validation.txt", "w") as f:
            f.write(str(full_text))
        print("Writing to:", os.path.abspath(f"{folder_path}/validation.txt"))
        return full_text
    except Exception as e:
        print("Error writing file:", e)
        return None


In [35]:
building_data_graph.bind('forc', 'https://w3id.org/b4b/schemes/algorithms/forc#')
shape_graph.bind('brick-ev', 'https://w3id.org/b4b/schemes/algorithms/brick-ev#')

In [36]:
building_data_graph.bind('forc', 'https://w3id.org/b4b/schemes/algorithms/forc#')
building_data_graph.bind('brick-ev', 'https://w3id.org/b4b/schemes/algorithms/brick-ev#')
res=validate_algorithm('dynamic/LoadForecastAlgorithm')
print(res)

Writing to: c:\Users\lasit\Documents\SoftwarePhD\mpc-schema-journal-paper\validation\building_v1\dynamic\LoadForecastAlgorithm\validation.txt
shape_graph_brick_Building_Electrical_Meter.ttl
Validation Report
Conforms: True

shape_graph_brick_Outside_Air_Temperature_Sensor.ttl
Validation Report
Conforms: False
Results (1):
Constraint Violation in SPARQLConstraintComponent (http://www.w3.org/ns/shacl#SPARQLConstraintComponent):
	Severity: sh:Violation
	Source Shape: :requiredClass_f6d49ea0-639c-4d2c-8724-05dfba5e7e1c
	Focus Node: <urn:algorithm_shapes#targetNode>
	Value Node: <urn:algorithm_shapes#targetNode>
	Source Constraint: [ rdf:type sh:SPARQLConstraint ; sh:message Literal("The building must contain at least one instance of the data point.") ; sh:select Literal("
            SELECT ?this
            WHERE {
                FILTER NOT EXISTS {
                    ?sensor a brick:Outside_Air_Temperature_Sensor  .
                }
            }
        ") ]
	Message: The building mu

In [37]:
building_data_graph.bind('forc', 'https://w3id.org/b4b/schemes/algorithms/forc#')
building_data_graph.bind('brick-ev', 'https://w3id.org/b4b/schemes/algorithms/brick-ev#')
res=validate_algorithm('dynamic/RealtimeFeedbackAlgorithm')
print(res)

Writing to: c:\Users\lasit\Documents\SoftwarePhD\mpc-schema-journal-paper\validation\building_v1\dynamic\RealtimeFeedbackAlgorithm\validation.txt
shape_graph_brick-ev_EV_Arrival_SOC.ttl
Validation Report
Conforms: False
Results (1):
Constraint Violation in SPARQLConstraintComponent (http://www.w3.org/ns/shacl#SPARQLConstraintComponent):
	Severity: sh:Violation
	Source Shape: :requiredClass_d4edee03-f735-4834-8f74-0ee8a1958c52
	Focus Node: <urn:algorithm_shapes#targetNode>
	Value Node: <urn:algorithm_shapes#targetNode>
	Source Constraint: [ rdf:type sh:SPARQLConstraint ; sh:message Literal("The building must contain at least one instance of the data point.") ; sh:select Literal("
            SELECT ?this
            WHERE {
                FILTER NOT EXISTS {
                    ?sensor a brick-ev:EV_Arrival_SOC  .
                }
            }
        ") ]
	Message: The building must contain at least one instance of the data point.

shape_graph_brick-ev_EV_Arrival_Time.ttl
Validation

In [38]:
res=validate_algorithm('dynamic/EVStateEstimationAlgorithm')
print(res)

Writing to: c:\Users\lasit\Documents\SoftwarePhD\mpc-schema-journal-paper\validation\building_v1\dynamic\EVStateEstimationAlgorithm\validation.txt
shape_graph_brick-ev_Charging_Session_Status.ttl
Validation Report
Conforms: False
Results (1):
Constraint Violation in SPARQLConstraintComponent (http://www.w3.org/ns/shacl#SPARQLConstraintComponent):
	Severity: sh:Violation
	Source Shape: :requiredClass_f71511cb-3126-4187-8f98-c31d12a01f96
	Focus Node: <urn:algorithm_shapes#targetNode>
	Value Node: <urn:algorithm_shapes#targetNode>
	Source Constraint: [ rdf:type sh:SPARQLConstraint ; sh:message Literal("The building must contain at least one instance of the data point.") ; sh:select Literal("
            SELECT ?this
            WHERE {
                FILTER NOT EXISTS {
                    ?sensor a brick-ev:Charging_Session_Status  .
                }
            }
        ") ]
	Message: The building must contain at least one instance of the data point.

shape_graph_brick-ev_EV_Arrival_

In [39]:
res=validate_algorithm('dynamic/EVChargingCurrentSetpointCommunication')
print(res)

Writing to: c:\Users\lasit\Documents\SoftwarePhD\mpc-schema-journal-paper\validation\building_v1\dynamic\EVChargingCurrentSetpointCommunication\validation.txt
shape_graph_brick-ev_EV_Electric_Energy_Meter.ttl
Validation Report
Conforms: True




In [40]:
res=validate_algorithm('dynamic/DynamicParameterEstimationAlgorithm')
print(res)

Writing to: c:\Users\lasit\Documents\SoftwarePhD\mpc-schema-journal-paper\validation\building_v1\dynamic\DynamicParameterEstimationAlgorithm\validation.txt
shape_graph_brick-ev_EV_Electric_Energy_Meter.ttl
Validation Report
Conforms: False
Results (1):
Constraint Violation in SPARQLConstraintComponent (http://www.w3.org/ns/shacl#SPARQLConstraintComponent):
	Severity: sh:Violation
	Source Shape: :requiredClass_e5e13c08-0176-4ad7-9d9e-a6e26dab0089
	Focus Node: <urn:algorithm_shapes#targetNode>
	Value Node: <urn:algorithm_shapes#targetNode>
	Source Constraint: [ rdf:type sh:SPARQLConstraint ; sh:message Literal("The building must contain at least one instance of the data point.") ; sh:select Literal("
            SELECT ?this
            WHERE {
                FILTER NOT EXISTS {
                    ?sensor a brick-ev:EV_Electric_Energy_Meter  .
                }
            }
        ") ]
	Message: The building must contain at least one instance of the data point.

shape_graph_brick-ev_